Considera el problema de la planificación semanal de la producción de una empresa de fabricación de automóviles. La empresa fabrica $M$ modelos de automóvil. La producción de estos modelos requiere la ejecución de $T$ tareas (por ejemplo, el ensamblaje de componentes mecánicos, pintura, perfilado, montaje de componentes eléctricas, etc.). La realización de estas tareas requiere personal cualificado, en un número de operarios que es diferente para cada modelo de automóvil. Sea $o_{mt},  m = 1, \ldots, M; t = 1, \ldots, T$, el número de operarios especializados en la tarea $t$ que se necesita para fabricar un automóvil modelo $m$ a la semana. Sabiendo que $\overline{o}_{t}$, con $t = 1, \ldots, T$ representa el número máximo de operarios cualificados para ejecutar la tarea $t$ de los que la empresa dispone en una semana laboral y que el beneficio que la empresa obtiene por la venta del modelo $m$ de automóvil es $b_m$, se pide:

1.   Construye un modelo de programación entera en AMPL para maximizar el beneficio semanal que hace la empresa a partir de la fabricación de los diferentes modelos de automóviles (4 puntos).
2.   Considera el caso particular en que la empresa fabrica tres modelos de automóviles ($M = 3$) cuyo montaje requiere de hasta cuatro tareas especializadas ($T = 4$). Los operarios disponibles para la ejecución de cada tarea, el número de ellos que requiere cada modelo de automóvil y los beneficios (en miles de €) que obtiene la empresa de la venta de cada modelo, son los especificados en la siguiente tabla.

<center>

|  Modelo    | Tarea 1   | Tarea 2  | Tarea 3 | Tarea 4 | Beneficio|
| ------------  | --------     | ---      | ---    |  ---    |--- |
| 1            | 1       | 1 | 1|  1 |  12
| 2            | 2       | 2 | 1| 3  |  20
| 3            | 1       | 2 | 3|  0  |  9
| **#Operarios**   | 7   | 10      | 8   |  6    |

</center>



*   Resuelve este caso particular con un solver para programas lineales enteros (1 punto).
*   Resuelve este caso particular mediante el *algoritmo de ramificación y acotación*, utilizando un solver adecuado para resolver los problemas lineales relajados que sean necesarios. Representa el árbol al que la aplicación del algoritmo ha dado lugar, especificando los tipos de podas que has realizado en cada caso hasta certificar la solución óptima del problema entero (5 puntos).
   





In [1]:
%pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.3 MB/s eta 0:00:00


In [2]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

In [82]:
FabAutomoviles = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


## Ejercicio 1

In [83]:
FabAutomoviles.eval(r"""

reset;
set BENEFICIOS;
set TAREAS;
param beneficios {BENEFICIOS} >= 0;
param n_max {TAREAS} >= 0;
param cont {BENEFICIOS, TAREAS} >= 0;


var x {BENEFICIOS} integer, >= 0;

maximize beneficio_total:
    sum{i in BENEFICIOS} beneficios[i]*x[i];
s.t. cumplir_tareas {j in TAREAS}:
    sum{i in BENEFICIOS} cont[i,j]*x[i] <= n_max[j];

""")

## Ejercicio 2

### Apartado 1

In [84]:
def preparar_datos_fabricacion():
    import pandas as pd
    import numpy as np

    beneficios_df = pd.DataFrame(
        [
            ("B1", 12),
            ("B2", 20),
            ("B3", 9)
        ],
        columns=["BENEFICIOS", "beneficios"],
    ).set_index("BENEFICIOS")


    tareas_df = pd.DataFrame(
        [
            ("T1", 7),
            ("T2", 10),
            ("T3", 8),
            ("T4", 6)
        ],
        columns=["TAREAS", "n_max"],
    ).set_index("TAREAS")

    cont_df = pd.DataFrame(
        np.array(
            [
                [1, 1, 1, 1],
                [2, 2, 1, 3],
                [1, 2, 3, 0]
            ]
        ),
        columns= tareas_df.index.to_list(),
        index  = beneficios_df.index.to_list(),
    )



    return beneficios_df, tareas_df, cont_df

In [85]:
beneficios_df, tareas_df, cont_df = preparar_datos_fabricacion()

FabAutomoviles.set_data(beneficios_df, "BENEFICIOS")
FabAutomoviles.set_data(tareas_df, "TAREAS")
FabAutomoviles.get_parameter("cont").set_values(cont_df)


In [86]:
FabAutomoviles.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert FabAutomoviles.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 72
2 simplex iterations


In [87]:
x = FabAutomoviles.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
beneficio_total = FabAutomoviles.get_objective("beneficio_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Beneficio total: {beneficio_total:.3f}")

    x.val
B1      6
B2      0
B3      0
Beneficio total: 72.000


### Apartado 2

In [25]:
FabAutomovilesR = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [26]:
FabAutomovilesR.eval(r"""

reset;
set BENEFICIOS;
set TAREAS;
param beneficios {BENEFICIOS} >= 0;
param n_max {TAREAS} >= 0;
param cont {BENEFICIOS, TAREAS} >= 0;


var x {BENEFICIOS} >= 0;

maximize beneficio_total:
    sum{i in BENEFICIOS} beneficios[i]*x[i];
s.t. cumplir_tareas {j in TAREAS}:
    sum{i in BENEFICIOS} cont[i,j]*x[i] <= n_max[j];

""")

In [27]:
beneficios_df, tareas_df, cont_df = preparar_datos_fabricacion()

FabAutomovilesR.set_data(beneficios_df, "BENEFICIOS")
FabAutomovilesR.set_data(tareas_df, "TAREAS")
FabAutomovilesR.get_parameter("cont").set_values(cont_df)

In [28]:
FabAutomovilesR.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert FabAutomovilesR.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 78
3 simplex iterations


In [35]:
x = FabAutomovilesR.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
beneficio_total = FabAutomovilesR.get_objective("beneficio_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Beneficio total: {beneficio_total:.3f}")

    x.val
B1  6.000
B2  0.000
B3  0.667
Beneficio total: 78.000


Solución del problema relajado. Acotamos superiormente el valor objetivo del problema entero por 78.

In [36]:
P1 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [37]:
P1.eval(r"""

reset;
set BENEFICIOS;
set TAREAS;
param beneficios {BENEFICIOS} >= 0;
param n_max {TAREAS} >= 0;
param cont {BENEFICIOS, TAREAS} >= 0;


var x {BENEFICIOS} >= 0;

maximize beneficio_total:
    sum{i in BENEFICIOS} beneficios[i]*x[i];
s.t. cumplir_tareas {j in TAREAS}:
    sum{i in BENEFICIOS} cont[i,j]*x[i] <= n_max[j];
    bounding: x["B3"] <= 0;

""")

In [40]:
beneficios_df, tareas_df, cont_df = preparar_datos_fabricacion()

P1.set_data(beneficios_df, "BENEFICIOS")
P1.set_data(tareas_df, "TAREAS")
P1.get_parameter("cont").set_values(cont_df)

In [41]:
P1.solve(solver="cplex") # Resolvemos con el solver "cplex"

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 72
1 simplex iterations


In [42]:
x = P1.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
beneficio_total = P1.get_objective("beneficio_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Beneficio total: {beneficio_total:.3f}")

    x.val
B1      6
B2      0
B3      0
Beneficio total: 72.000


Podamos P1 por integridad, acotamos inferiormente el valor de la función objetivo del problema entero por 72.

In [43]:
P2 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [44]:
P2.eval(r"""

reset;
set BENEFICIOS;
set TAREAS;
param beneficios {BENEFICIOS} >= 0;
param n_max {TAREAS} >= 0;
param cont {BENEFICIOS, TAREAS} >= 0;


var x {BENEFICIOS} >= 0;

maximize beneficio_total:
    sum{i in BENEFICIOS} beneficios[i]*x[i];
s.t. cumplir_tareas {j in TAREAS}:
    sum{i in BENEFICIOS} cont[i,j]*x[i] <= n_max[j];
    bounding: x["B3"] >= 1;

""")

In [45]:
beneficios_df, tareas_df, cont_df = preparar_datos_fabricacion()

P2.set_data(beneficios_df, "BENEFICIOS")
P2.set_data(tareas_df, "TAREAS")
P2.get_parameter("cont").set_values(cont_df)

In [46]:
P2.solve(solver="cplex") # Resolvemos con el solver "cplex"

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 73
3 simplex iterations


In [47]:
x = P2.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
beneficio_total = P2.get_objective("beneficio_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Beneficio total: {beneficio_total:.3f}")

    x.val
B1    4.5
B2    0.5
B3    1.0
Beneficio total: 73.000


Solución de P2. El valor de la función objetivo es 73, podemos ramificar.

In [60]:
P3 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [62]:
P3.eval(r"""

reset;
set BENEFICIOS;
set TAREAS;
param beneficios {BENEFICIOS} >= 0;
param n_max {TAREAS} >= 0;
param cont {BENEFICIOS, TAREAS} >= 0;


var x {BENEFICIOS} >= 0;

maximize beneficio_total:
    sum{i in BENEFICIOS} beneficios[i]*x[i];
s.t. cumplir_tareas {j in TAREAS}:
    sum{i in BENEFICIOS} cont[i,j]*x[i] <= n_max[j];
    bounding1: x["B3"] >= 1;
    bounding2: x["B1"] <= 4;

""")

In [63]:
beneficios_df, tareas_df, cont_df = preparar_datos_fabricacion()

P3.set_data(beneficios_df, "BENEFICIOS")
P3.set_data(tareas_df, "TAREAS")
P3.get_parameter("cont").set_values(cont_df)

In [64]:
P3.solve(solver="cplex") # Resolvemos con el solver "cplex"

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 71.33333333
3 simplex iterations


In [65]:
x = P3.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
beneficio_total = P3.get_objective("beneficio_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Beneficio total: {beneficio_total:.3f}")

    x.val
B1  4.000
B2  0.667
B3  1.111
Beneficio total: 71.333


Podamos P3 por acotación, ya que el valor de la función objetivo para este problema es menor que 72.

In [69]:
P4 = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


In [70]:
P4.eval(r"""

reset;
set BENEFICIOS;
set TAREAS;
param beneficios {BENEFICIOS} >= 0;
param n_max {TAREAS} >= 0;
param cont {BENEFICIOS, TAREAS} >= 0;


var x {BENEFICIOS} >= 0;

maximize beneficio_total:
    sum{i in BENEFICIOS} beneficios[i]*x[i];
s.t. cumplir_tareas {j in TAREAS}:
    sum{i in BENEFICIOS} cont[i,j]*x[i] <= n_max[j];
    bounding1: x["B3"] >= 1;
    bounding2: x["B1"] >= 5;

""")

In [71]:
beneficios_df, tareas_df, cont_df = preparar_datos_fabricacion()

P4.set_data(beneficios_df, "BENEFICIOS")
P4.set_data(tareas_df, "TAREAS")
P4.get_parameter("cont").set_values(cont_df)

In [72]:
P4.solve(solver="cplex") # Resolvemos con el solver "cplex"

Solution determined by presolve;
objective beneficio_total = 69.


In [73]:
x = P4.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
beneficio_total = P4.get_objective("beneficio_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Beneficio total: {beneficio_total:.3f}")

    x.val
B1      5
B2      0
B3      1
Beneficio total: 69.000


Podamos P4 por acotación, ya que el valor de la función objetivo para este problema es menor que 72.

  La solución de nuestro problema entero viene dada por resolver el problema P1, obteniendo x = (6,0,0) con valor objetivo z = 72, justamente la solución proporcionada por el solver para el problema con las variables restringidas a ser números enteros.